In [0]:
from keras.models import Model
from keras.layers import Flatten, Dense, Input, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.engine.topology import get_source_inputs
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers.core import Dropout, Lambda
from keras.layers import merge
import warnings
from keras.layers.core import Activation
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.datasets import cifar10
import cv2
import numpy as np

# (x_train, y_train), (x_test, y_test) = cifar100.load_data()
# y_train = np_utils.to_categorical(y_train, 100)
# y_test = np_utils.to_categorical(y_test, 100)
# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train  /= 255
# x_test /= 255

# slim = tf.contrib.slim

In [0]:
import tensorflow as tf
slim = tf.contrib.slim
trunc_normal = lambda stddev: tf.truncated_normal_initializer(0.0, stddev)

In [0]:
def inception_v3_base(inputs,
                      final_endpoint='Mixed_7c',
                      min_depth=16,
                      depth_multiplier=1.0,
                      scope=None):
  
  end_points = {}
  
  if depth_multiplier <= 0:
    raise ValueError('depth_multiplier is not greater than zero.')
  depth = lambda d: max(int(d * depth_multiplier), min_depth)

  with tf.variable_scope(scope, 'InceptionV3', [inputs]):
    with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d],
                        stride=1, padding='VALID'):
      
      # 299 x 299 x 3
      end_point = 'Conv2d_1a_3x3'
      net = slim.conv2d(inputs, depth(32), [3, 3], strides=2, scope=end_point)
      end_points[end_point] = net
      if end_point == final_endpoint: return net, end_points
      # 149 x 149 x 32
      end_point = 'Conv2d_2a_3x3'
      net = slim.conv2d(net, depth(32), [3, 3], scope=end_point)
      end_points[end_point] = net
      if end_point == final_endpoint: return net, end_points
      # 147 x 147 x 32
      end_point = 'Conv2d_2b_3x3'
      net = slim.conv2d(net, depth(64), [3, 3], padding='SAME', scope=end_point)
      end_points[end_point] = net
      if end_point == final_endpoint: return net, end_points
      # 147 x 147 x 64
      end_point = 'MaxPool_3a_3x3'
      net = slim.max_pool2d(net, [3, 3], stride=2, scope=end_point)
      end_points[end_point] = net
      if end_point == final_endpoint: return net, end_points
      # 73 x 73 x 64
      end_point = 'Conv2d_3b_1x1'
      net = slim.conv2d(net, depth(80), [1, 1], scope=end_point)
      end_points[end_point] = net
      if end_point == final_endpoint: return net, end_points
      # 73 x 73 x 80
      end_point = 'Conv2d_4a_3x3'
      net = slim.conv2d(net, depth(192), [3, 3], scope=end_point)
      end_points[end_point] = net
      if end_point == final_endpoint: return net, end_points
      # 71 x 71 x 192
      end_point = 'MaxPool_5a_3x3'
      net = slim.max_pool2d(net, [3, 3], stride=2, scope=end_point)
      end_points[end_point] = net
      if end_point == final_endpoint: return net, end_points
      # 35 x 35 x 192
      
    with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d], stride=1, padding='SAME'):
      # mixed: 35 x 35 x 256
      end_point = 'Mixed_5b'
      with tf.variable_scope(end_point):
        with tf.variable_scope('Branch_0'):
          branch_0 = slim.conv2d(net, depth(64), [1, 1], scope='Conv2d_0a_1x1')
        with tf.variable_scope('Branch_1'):
          branch_1 = slim.conv2d(net, depth(48), [1, 1], scope='Conv2d_0a_1x1')
          branch_1 = slim.conv2d(branch_1, depth(64), [5, 5], scope='Conv2d_0b_5x5')
        with tf.variable_scope('Branch_2'):
          branch_2 = slim.conv2d(net, depth(64), [1, 1], scope='Conv2d_0a_1x1')
          branch_2 = slim.conv2d(branch_2, depth(96), [3, 3], scope='Conv2d_0b_3x3')
          branch_2 = slim.conv2d(branch_2, depth(96), [3, 3], scope='Conv2d_0c_3x3')
        with tf.variable_scope('Branch_3'):
          branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
          branch_3 = slim.conv2d(branch_3, depth(32), [1, 1], scope='Conv2d_0b_1x1')
        net = tf.concat(axis=3, values=[branch_0, branch_1, branch_2, branch_3])
      end_points[end_point] = net
      if end_point == final_endpoint: 
        return net, end_points
      
      # mixed_1: 35 x 35 x 288
      end_point = 'Mixed_5c'
      with tf.variable_scope(end_point):
        with tf.variable_scope('Branch_0'):
          branch_0 = slim.conv2d(net, depth(64), [1, 1], scope='Conv2d_0a_1x1')
        with tf.variable_scope('Branch_1'):
          branch_1 = slim.conv2d(net, depth(48), [1, 1], scope='Conv2d_0a_1x1')
          branch_1 = slim.conv2d(branch_1, depth(64), [5, 5], scope='Conv2d_0b_5x5')
        with tf.variable_scope('Branch_2'):
          branch_2 = slim.conv2d(net, depth(64), [1, 1], scope='Conv2d_0a_1x1')
          branch_2 = slim.conv2d(branch_2, depth(96), [3, 3], scope='Conv2d_0b_3x3')
          branch_2 = slim.conv2d(branch_2, depth(96), [3, 3], scope='Conv2d_0c_3x3')
        with tf.variable_scope('Branch_3'):
          branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
          branch_3 = slim.conv2d(branch_3, depth(64), [1, 1], scope='Conv2d_0b_1x1')
        net = tf.concat(axis=3, value=[branch_0, branch_1, branch_2, branch_3])
      end_points[end_point] = net
      if end_point == final_endpoint: return net, end_points
      
      # mixed_2: 35 x 35 x 288
      end_point = 'Mixed_5d'
      with tf.variable_scope(end_point):
        with tf.variable_scope('Branch_0'):
          branch_0 = slim.conv2d(net, depth(64), [1, 1], scope='Conv2d_0a_1x1')
        with tf.variable_scope('Branch_1'):
          branch_1 = slim.conv2d(net, depth(48), [1, 1], scope='Conv2d_0a_1x1')
          branch_1 = slim.conv2d(branch_1, depth(64), [5, 5], scope='Conv2d_0b_5x5')
        with tf.variable_scope('Branch_2'):
          branch_2 = slim.conv2d(net, depth(64), [1, 1], scope='Conv2d_0a_1x1')
          branch_2 = slim.conv2d(branch_2, depth(96), [3, 3], scope='Conv2d_0b_3x3')
          branch_2 = slim.conv2d(branch_2, depth(96), [3, 3], scope='Conv2d_0c_3x3')
        with tf.variable_scope('Branch_3'):
          branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
          branch_3 = slim.conv2d(branch_3, depth(64), [1, 1], scope='Conv2d_0b_1x1')
        net = tf.concat(axis=3, values=[branch_0, branch_1, branch_2, branch_3])
      end_points[end_point] = net
      if end_point == final_endpoint: return net, end_points
      
      # mixed_3 : 17 x 17 x 768
      end_point = 'Mixed_6a'
      with tf.variable_scope(end_point):
        with tf.variable_scope('Branch_0'):
          branch_0 = slim.conv2d(net, depth(384), [3, 3], stride=2, padding='VALID', scope='Conv2d_1a_1x1')
        with tf.variable_scope('Branch_1'):
          branch_1 = slim.conv2d(net, depth(64), [1, 1], scope='Conv2d_0a_1x1')
          branch_1 = slim.conv2d(branch_1, depth(96), [3, 3], scope='Conv2d_0b_3x3')
          branch_1 = slim.conv2d(branch_1, depth(96), [3, 3], stride=2, padding='VALID', scope='Conv2d_1a_1x1')
        with tf.variable_scope('Branch_2'):
          branch_2 = slim.max_pool2d(net, [3, 3], stride=2, padding='VALID', scope='MaxPool_1a_3x3')
        net = tf.concat(axis=3, values=[branch_0, branch_1, branch_2])
      end_points[end_point] = net
      if end_point == final_endpoint: return net, end_points
      
      # mixed_4 : 17 x 17 x 768
      end_point = 'Mixed_6b'
      with tf.variable_scope(end_point):
        with tf.variable_scope('Branch_0'):
          branch_0 = slim.conv2d(net, depth(192), [1, 1], scope='Conv2d_0a_1x1')
        with tf.variable_scope('Branch_1'):
          branch_1 = slim.conv2d(net, depth(128), [1, 1], scope='Conv2d_0a_1x1')
          branch_1 = slim.conv2d(branch_1, depth(128), [1, 7], scope='Conv2d_0b_1x7')
          branch_1 = slim.conv2d(branch_1, depth(192), [7, 1], scope='Conv2d_0c_7x1')
        with tf.variable_scope('Branch_2'):
          branch_2 = slim.conv2d(net, depth(128), [1, 1], scope='Conv2d_0a_1x1')
          branch_2 = slim.conv2d(branch_2, depth(128), [7, 1], scope='Conv2d_0b_7x1')
          branch_2 = slim.conv2d(branch_2, depth(128), [1, 7], scope='Conv2d_0c_1x7')
          branch_2 = slim.conv2d(branch_2, depth(128), [7, 1], scope='Conv2d_0d_7x1')
          branch_2 = slim.conv2d(branch_2, depth(128), [1, 7], scope='Conv2d_0e_1x7')
        with tf.variable_scope('Branch_3'):
          branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
          branch_3 = slim.conv2d(branch_3, depth(192), [1, 1], scope='Conv2d_0b_1x1')
        net = tf.concat(axis=3, values=[branch_0, branch_1, branch_2, branch_3])
      end_points[end_point] = net
      if end_point == final_endpoint: return net, end_points
      
      # mixed_5 : 17 x 17 x 768
      end_point = 'Mixed_6c'
      with tf.variable_scope(end_point):
        with tf.variable_scope('Branch_0'):
          branch_0 = slim.conv2d(net, depth(192), [1, 1], scope='Conv2d_0a_1x1')
        with tf.variable_scope('Branch_1'):
          branch_1 = slim.conv2d(net, depth(160), [1, 1], scope='Conv2d_0a_1x1')
          branch_1 = slim.conv2d(branch_1, depth(160), [1, 7], scope='Conv2d_0b_1x7')
          branch_1 = slim.conv2d(branch_1, depth(192), [7, 1], scope='Conv2d_0c_7x1')
        with tf.variable_scope('Branch_2'):
          branch_2 = slim.conv2d(net, depth(160), [1, 1], scope='Conv2d_0a_1x1')
          branch_2 = slim.conv2d(branch_2, depth(160), [7, 1], scope='Conv2d_0b_7x1')
          branch_2 = slim.conv2d(branch_2, depth(160), [1, 7], scope='Conv2d_0c_1x7')
          branch_2 = slim.conv2d(branch_2, depth(160), [7, 1], scope='Conv2d_0d_7x1')
          branch_2 = slim.conv2d(branch_2, depth(160), [1, 7], scope='Conv2d_0e_1x7')
        with tf.variable_scope('Branch_3'):
          branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
          branch_3 = slim.conv2d(branch_3, depth(192), [1, 1], scope='Conv2d_0b_1x1')
        net = tf.concat(axis=3, values=[branch_0, branch_1, branch_2, branch_3])
      end_points[end_point] = net
      if end_point == final_endpoint: return net, end_points
      
      # mixed_6 : 17 x 17 x 768
      end_point = 'Mixed_6d'
      with tf.variable_scope(end_point):
        with tf.variable_scope('Branch_0'):
          branch_0 = slim.conv2d(net, depth(192), [1, 1], scope='Conv2d_0a_1x1')
        with tf.variable_scope('Branch_1'):
          branch_1 = slim.conv2d(net, depth(160), [1, 1], scope='Conv2d_0a_1x1')
          branch_1 = slim.conv2d(branch_1, depth(160), [1, 7], scope='Conv2d_0b_1x7')
          branch_1 = slim.conv2d(branch_1, depth(160), [7, 1], scope='Conv2d_0c_7x1')
        with tf.variable_scope('Branch_2'):
          branch_2 = slim.conv2d(net, depth(160), [1, 1], scope='Conv2d_0a_1x1')
          branch_2 = slim.conv2d(branch_2, depth(160), [7, 1], scope='Conv2d_0b_7x1')
          branch_2 = slim.conv2d(branch_2, depth(160), [1, 7], scope='Conv2d_0c_1x7')
          branch_2 = slim.conv2d(branch_2, depth(160), [7, 1], scope='Conv2d_0d_7x1')
          branch_2 = slim.conv2d(branch_2, depth(160), [1, 7], scope='Conv2d_0e_1x7')
        with tf.variable_scope('Branch_3'):
          branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
          branch_3 = slim.conv2d(branch_3, depth(192), [1, 1], scope='Conv2d_0b_1x1')
        net = tf.concat(axis=3, values=[branch_0, branch_1, branch_2, branch_3])
      end_points[end_point] = net
      if end_point == final_endpoint: return net, end_points
      
      # mixed_7 : 17 x 17 x 768
      end_point = 'Mixed_6e'
      with tf.variable_scope(end_point):
        with tf.variable_scope('Branch_0'):
          branch_0 = slim.conv2d(net, depth(192), [1, 1], scope='Conv2d_0a_1x1')
        with tf.variable_scope('Branch_1'):
          branch_1 = slim.conv2d(net, depth(192), [1, 1], scope='Conv2d_0a_1x1')
          branch_1 = slim.conv2d(branch_1, depth(192), [1, 7], scope='Conv2d_0b_1x7')
          branch_1 = slim.conv2d(branch_1, depth(192), [7, 1], scopee='Conv2d_0c_7x1')
        with tf.variable_scope('Branch_2'):
          branch_2 = slim.conv2d(net, depth(192), [1, 1], scope='Conv2d_0a_1x1')
          branch_2 = slim.conv2d(branch_2, depth(192), [7, 1], scope='Conv2d_0b_7x1')
          branch_2 = slim.conv2d(branch_2, depth(192), [1, 7], scope='Conv2d_0c_1x7')
          branch_2 = slim.conv2d(branch_2, depth(192), [7, 1], scope='Conv2d_0d_7x1')
          branch_2 = slim.conv2d(branch_2, depth(192), [1, 7], scope='Conv2d_0e_1x7')
        with tf.variable_scope('Branch_3'):
          branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
          branch_3 = slim.conv2d(branch_3, depth(192), [1, 1], scopee='Conv2d_0b_1x1')
        net = tf.concat(axis=3, values=[branch_0, branch_1, branch_2, branch_3])
      end_points[end_point] = net
      if end_point == final_endpoint: return net, end_points
      
      # mixed_8: 8 x  8 x 1280
      end_point = 'Mixed_7a'
      with tf.variable_scope(end_point):
        with tf.variable_scope('Branch_0'):
          branch_0 = slim.conv2d(net, depth(192), [1, 1], scope='Conv2d_0a_1x1')
          branch_0 = slim.conv2d(branch_0, depth(320), [3, 3], stride=2, padding='VALID', scope='Conv2d_1a_3x3')
        with tf.variable_scope('Branch_1'):
          branch_1 = slim.conv2d(net, depth(192), [1, 1], scope='Conv2d_0a_1x1')
          branch_1 = slim.conv2d(branch_1, depth(192), [1, 7], scope='Conv2d_0b_1x7')
          branch_1 = slim.conv2d(branch_1, depth(192), [7, 1], scope='Conv2d_0c_7x1')
          branch_1 = slim.conv2d(branch_1, depth(192), [3, 3], stride=2, padding='VALID', scope='Conv2d_1a_3x3')
        with tf.variable_scope('Branch_2'):
          branch_2 = slim.max_pool2d(net, [3, 3], stride=2, padding='VALID', scope='MaxPool_1a_3x3')
        net = tf.concat(axis=3, values=[branch_0, branch_1, branch_2])
      end_points[end_point] = net
      if end_point == final_endpoint: return net, end_points
      
      # mixed_9: 8 x 8 x 2048
      end_point = 'Mixed_7b'
      with tf.variable_scope(end_point):
        with tf.variable_scope('Branch_0'):
          branch_0 = slim.conv2d(net, depth(320), [1, 1], scope='Conv2d_0a_1x1')
        with tf.variable_scope('Branch_1'):
          branch_1 = slim.conv2d(net, depth(384), [1, 1], scope='Conv2d_0a_1x1')
          branch_1 = tf.concat(axis=3, values=[
              slim.conv2d(branch_1, depth(384), [1, 3], scope='Conv2d_0b_1x3'),
              slim.conv2d(branch_1, depth(384), [3, 1], scope='Conv2d_0b_3x1')
          ])
        with tf.variable_scope('Branch_2'):
          branch_2 = slim.conv2d(net, depth(448), [1, 1], scope='Conv2d_0a_1x1')
          branch_2 = slim.conv2d(branch_2, depth(384), [3, 3], scope='Conv2d_0b_3x3')
          branch_2 =  tf.concat(axis=3, values=[
              slim.conv2d(branch_2, depth(384), [1, 3], scope='Conv2d_0c_1x3'),
              slim.conv2d(branch_2, deppth(384), [3, 1], scope='Conv2d_0d_3x1')
          ])
        with tf.variable_scope('Branch_3'):
          branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
          branch_3 = slim.conv2d(branch_3, depth(192), [1, 1], scope='Conv2d_0b_1x1')
        net = tf.concat(axis=3, values=[branch_0, branch_1, branch_2, branch_3])
      end_points[end_point] = net
      if end_point == final_endpoint: return net, end_points
      
      # mixed_10 : 8 x 8 x 2048
      end_point = 'Mixed_7c'
      with tf.variable_scope(end_point):
        with tf.variable_scope('Branch_0'):
          branch_0 = slim.conv2d(net, depth(320), [1, 1], scope='Conv2d_0a_1x1')
        with tf.variable_scope('Branch_1'):
          branch_1 = slim.conv2d(net, depth(384), [1, 1], scope='Conv2d_0a_1x1')
          branch_1 = tf.concat(axis=3, values=[
              slim.conv2d(branch_1, depth(384), [1, 3], scope='Conv2d_0a_1x3'),
              slim.conv2d(branch_1, depth(384), [3, 1], scope='Conv2d_0b_3x1')
          ])
        with tf.variable_scope('Branch_2'):
          branch_2 = slim.conv2d(net, depth(448), [1, 1], scope='Conv2d_0a_1x1')
          branch_2 = slim.conv2d(branch_2, depth(384), [3, 3], scope='Conv2d_0b_3x3')
          branch_2 = tf.concat(axis=3, values=[
              slim.conv2d(branch_2, depth(384), [1, 3], scope='Conv2d_0c_1x3'),
              slim.conv2d(branch_2, depth(384), [3, 1], scope='Conv2d_0d_3x1')
          ])
        with tf.variable_scope('Branch_3'):
          branch_3 = slim.avg_pool2d(net, [3, 3], scope='AvgPool_0a_3x3')
          branch_3 = slim.conv2d(branch_3, depth(192), [1, 1], scope='Conv2d_0b_1x1')
        net = tf.concat(axis=3, values=[branch_0, branch_1, branch_2, branch_3])
      end_points[end_point] = net
      if end_point == final_endpoint: return net, end_points
    raise ValueError('Unknown final endpoint %s' % final_endpoint)

def inception_v3(inputs,
                 num_classes=1000,
                 is_training=True,
                 dropout_keep_prob=0.8,
                 min_depth=16,
                 depth_multiplier=1.0,
                 prediction_fn=slim.softmax,
                 spatial_squeeze=True,
                 reuse=None,
                 create_aux_logits=True,
                 scope='InceptionV3',
                 global_pool=False):

  if depth_multiplier <= 0:
    raise ValueError('depth_multiplier is not greater than zero.')
  depth = lambda d: max(int(d * depth_multiplier), min_depth)

  with tf.variable_scope(scope, 'InceptionV3', [inputs], reuse=reuse) as scope:
    with slim.arg_scope([slim.batch_norm, slim.dropout],
                        is_training=is_training):
      net, end_points = inception_v3_base(
          inputs, scope=scope, min_depth=min_depth,
          depth_multiplier=depth_multiplier)

      if create_aux_logits and num_classes:
        with slim.arg_scope([slim.conv2d, slim.max_pool2d, slim.avg_pool2d],
                            stride=1, padding='SAME'):
          aux_logits = end_points['Mixed_6e']
          with tf.variable_scope('AuxLogits'):
            aux_logits = slim.avg_pool2d(
                aux_logits, [5, 5], stride=3, padding='VALID',
                scope='AvgPool_1a_5x5')
            aux_logits = slim.conv2d(aux_logits, depth(128), [1, 1],
                                     scope='Conv2d_1b_1x1')

            kernel_size = _reduced_kernel_size_for_small_input(
                aux_logits, [5, 5])
            aux_logits = slim.conv2d(
                aux_logits, depth(768), kernel_size,
                weights_initializer=trunc_normal(0.01),
                padding='VALID', scope='Conv2d_2a_{}x{}'.format(*kernel_size))
            aux_logits = slim.conv2d(
                aux_logits, num_classes, [1, 1], activation_fn=None,
                normalizer_fn=None, weights_initializer=trunc_normal(0.001),
                scope='Conv2d_2b_1x1')
            if spatial_squeeze:
              aux_logits = tf.squeeze(aux_logits, [1, 2], name='SpatialSqueeze')
            end_points['AuxLogits'] = aux_logits

      with tf.variable_scope('Logits'):
        if global_pool:
          net = tf.reduce_mean(net, [1, 2], keep_dims=True, name='GlobalPool')
          end_points['global_pool'] = net
        else:
          kernel_size = _reduced_kernel_size_for_small_input(net, [8, 8])
          net = slim.avg_pool2d(net, kernel_size, padding='VALID',
                                scope='AvgPool_1a_{}x{}'.format(*kernel_size))
          end_points['AvgPool_1a'] = net
        if not num_classes:
          return net, end_points
        # 1 x 1 x 2048
        net = slim.dropout(net, keep_prob=dropout_keep_prob, scope='Dropout_1b')
        end_points['PreLogits'] = net
        # 2048
        logits = slim.conv2d(net, num_classes, [1, 1], activation_fn=None,
                             normalizer_fn=None, scope='Conv2d_1c_1x1')
        if spatial_squeeze:
          logits = tf.squeeze(logits, [1, 2], name='SpatialSqueeze')
        # 1000
      end_points['Logits'] = logits
      end_points['Predictions'] = prediction_fn(logits, scope='Predictions')
  return logits, end_points
inception_v3.default_image_size = 299    
    
def inception_arg_scope(weight_decay=0.00004,
                        use_batch_norm=True,
                        batch_norm_decay=0.9997,
                        batch_norm_epsilon=0.001,
                        activation_fn=tf.nn.relu):
  batch_norm_params = {
      'decay': batch_norm_decay,
      'epsilon': batch_norm_epsilon,
      'updates_collections': tf.GraphKeys.UPDATE_OPS,
      'fused': None,
  }
  if use_batch_norm:
    normalizer_fn = slim.batch_norm
    normalizer_params = batch_norm_params
  else:
    normalizer_fn = None
    normalizer_params = {}

  with slim.arg_scope([slim.conv2d, slim.fully_connected],
                      weights_regularizer=slim.l2_regularizer(weight_decay)):
    with slim.arg_scope(
        [slim.conv2d],
        weights_initializer=slim.variance_scaling_initializer(),
        activation_fn=activation_fn,
        normalizer_fn=normalizer_fn,
        normalizer_params=normalizer_params) as sc:
      return sc
          

In [0]:
def conv2d_bn(x, nb_filter, nb_row, nb_col,
              border_mode='same', subsample=(1, 1),
              batch_norm=True, activation='relu',
              weight_decay=0, name=None):
    if name is not None:
        bn_name = 'bn_' + name
        conv_name = 'conv_' + name
    else:
        bn_name = None
        conv_name = None
    if K.image_dim_ordering() == 'th':
        bn_axis = 1
    else:
        bn_axis = 3

    if weight_decay and weight_decay > 0:
        x = Convolution2D(nb_filter, nb_row, nb_col,
                          subsample=subsample,
                          activation='relu',
                          W_regularizer=l2(weight_decay),
                          border_mode=border_mode,
                          name=conv_name)(x)
    else:
        x = Convolution2D(nb_filter, nb_row, nb_col,
                          subsample=subsample,
                          activation='relu',
                          border_mode=border_mode,
                          name=conv_name)(x)

    if batch_norm:
        x = BatchNormalization(axis=bn_axis, name=bn_name)(x)

    if activation:
        x = Activation(activation)(x)

    return x

In [0]:
def Inception_V3(include_top=True, weights='imagenet',
                input_tensor=None, input_shape=None,
                weight_decay=0.00004, num_classes=1000,
                dropout_prob=0., aux_include=True):

    if input_shape is None:
        input_shape = (299, 299)

    if K.image_dim_ordering() == 'th':
        input_shape = (3,) + input_shape
        channel_axis = 1
    else:
        input_shape = input_shape + (3,)
        channel_axis = 3

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        img_input = input_tensor

    # Using `tf` order
    # 299 x 299 x 3
    x = conv2d_bn(img_input, 32, 3, 3, subsample=(2, 2),
                  border_mode='valid', weight_decay=weight_decay,
                  name='0')

    # 149 x 149 x 32
    x = conv2d_bn(x, 32, 3, 3, border_mode='valid',
                  weight_decay=weight_decay, name='1')

    # 147 x 147 x 32
    x = conv2d_bn(x, 64, 3, 3, weight_decay=weight_decay, name='2')

    # 147 x 147 x 64
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool_1')(x)

    # 73  x 73 x 64
    x = conv2d_bn(x, 80, 1, 1, weight_decay=weight_decay, name='3')

    # 73 x 73 x 80
    x = conv2d_bn(x, 192, 3, 3, border_mode='valid',
                  weight_decay=weight_decay, name='4')

    # 71 x 71 x 192
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool_2')(x)

    # 35 x 35 x 192
    # mixed 0: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1, weight_decay=weight_decay)

    branch5x5 = conv2d_bn(x, 48, 1, 1, weight_decay=weight_decay)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5, weight_decay=weight_decay)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1, weight_decay=weight_decay)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3, weight_decay=weight_decay)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3, weight_decay=weight_decay)

    branch_pool = AveragePooling2D(
        (3, 3), strides=(1, 1), border_mode='same')(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1, weight_decay=weight_decay)

    x = merge([branch1x1, branch5x5, branch3x3dbl, branch_pool],
              mode='concat', concat_axis=channel_axis,
              name='mixed_0')

    for i in range(2):
        branch1x1 = conv2d_bn(x, 64, 1, 1, weight_decay=weight_decay)

        branch5x5 = conv2d_bn(x, 48, 1, 1, weight_decay=weight_decay)
        branch5x5 = conv2d_bn(branch5x5, 64, 5, 5, weight_decay=weight_decay)

        branch3x3dbl = conv2d_bn(x, 64, 1, 1, weight_decay=weight_decay)
        branch3x3dbl = conv2d_bn(
            branch3x3dbl, 96, 3, 3, weight_decay=weight_decay)
        branch3x3dbl = conv2d_bn(
            branch3x3dbl, 96, 3, 3, weight_decay=weight_decay)

        branch_pool = AveragePooling2D(
            (3, 3), strides=(1, 1), border_mode='same')(x)
        branch_pool = conv2d_bn(
            branch_pool, 64, 1, 1, weight_decay=weight_decay)

        x = merge([branch1x1, branch5x5, branch3x3dbl, branch_pool],
                  mode='concat', concat_axis=channel_axis,
                  name='mixed_' + str(i + 1))

    # mixed_3: 17 x 17 x 768
    branch3x3 = conv2d_bn(
        x, 384, 3, 3, subsample=(2, 2),
        border_mode='valid', weight_decay=weight_decay)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1, weight_decay=weight_decay)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 3, weight_decay=weight_decay)
    branch3x3dbl = conv2d_bn(
        branch3x3dbl, 96, 3, 3, subsample=(2, 2),
        border_mode='valid', weight_decay=weight_decay)

    branch_pool = MaxPooling2D((3, 3), strides=(2, 2), border_mode='valid')(x)
    x = merge([branch3x3, branch3x3dbl, branch_pool], mode='concat',
              concat_axis=channel_axis, name='mixed_3')

    # mixed_4: 17 x 17 x 768
    branch1x1 = conv2d_bn(x, 192, 1, 1, weight_decay=weight_decay)

    branch7x7 = conv2d_bn(x, 128, 1, 1, weight_decay=weight_decay)
    branch7x7 = conv2d_bn(branch7x7, 128, 1, 7, weight_decay=weight_decay)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1, weight_decay=weight_decay)

    branch7x7dbl = conv2d_bn(x, 128, 1, 1, weight_decay=weight_decay)
    branch7x7dbl = conv2d_bn(
        branch7x7dbl, 128, 7, 1, weight_decay=weight_decay)
    branch7x7dbl = conv2d_bn(
        branch7x7dbl, 128, 1, 7, weight_decay=weight_decay)
    branch7x7dbl = conv2d_bn(
        branch7x7dbl, 128, 7, 1, weight_decay=weight_decay)
    branch7x7dbl = conv2d_bn(
        branch7x7dbl, 192, 1, 7, weight_decay=weight_decay)

    branch_pool = AveragePooling2D(
        (3, 3), strides=(1, 1), border_mode='same')(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1, weight_decay=weight_decay)

    x = merge([branch1x1, branch7x7, branch7x7dbl, branch_pool],
              mode='concat', concat_axis=channel_axis, name='mixed_4')

    # mixed 5, 6: 17 x 17 x 768
    for i in range(2):
        branch1x1 = conv2d_bn(x, 192, 1, 1, weight_decay=weight_decay)

        branch7x7 = conv2d_bn(x, 160, 1, 1, weight_decay=weight_decay)
        branch7x7 = conv2d_bn(branch7x7, 160, 1, 7, weight_decay=weight_decay)
        branch7x7 = conv2d_bn(branch7x7, 192, 7, 1, weight_decay=weight_decay)

        branch7x7dbl = conv2d_bn(x, 160, 1, 1, weight_decay=weight_decay)
        branch7x7dbl = conv2d_bn(
            branch7x7dbl, 160, 7, 1, weight_decay=weight_decay)
        branch7x7dbl = conv2d_bn(
            branch7x7dbl, 160, 1, 7, weight_decay=weight_decay)
        branch7x7dbl = conv2d_bn(
            branch7x7dbl, 160, 7, 1, weight_decay=weight_decay)
        branch7x7dbl = conv2d_bn(
            branch7x7dbl, 192, 1, 7, weight_decay=weight_decay)

        branch_pool = AveragePooling2D(
            (3, 3), strides=(1, 1), border_mode='same')(x)
        branch_pool = conv2d_bn(
            branch_pool, 192, 1, 1, weight_decay=weight_decay)

        x = merge([branch1x1, branch7x7, branch7x7dbl, branch_pool],
                  mode='concat', concat_axis=channel_axis,
                  name='mixed_' + str(i + 5))

    # mixed 7: 17 x 17 x 768
    branch1x1 = conv2d_bn(x, 192, 1, 1, weight_decay=weight_decay)

    branch7x7 = conv2d_bn(x, 192, 1, 1, weight_decay=weight_decay)
    branch7x7 = conv2d_bn(branch7x7, 192, 1, 7, weight_decay=weight_decay)
    branch7x7 = conv2d_bn(branch7x7, 192, 7, 1, weight_decay=weight_decay)

    branch7x7dbl = conv2d_bn(x, 192, 1, 1, weight_decay=weight_decay)
    branch7x7dbl = conv2d_bn(
        branch7x7dbl, 192, 7, 1, weight_decay=weight_decay)
    branch7x7dbl = conv2d_bn(
        branch7x7dbl, 192, 1, 7, weight_decay=weight_decay)
    branch7x7dbl = conv2d_bn(
        branch7x7dbl, 192, 7, 1, weight_decay=weight_decay)
    branch7x7dbl = conv2d_bn(
        branch7x7dbl, 192, 1, 7, weight_decay=weight_decay)

    branch_pool = AveragePooling2D(
        (3, 3), strides=(1, 1), border_mode='same')(x)
    branch_pool = conv2d_bn(branch_pool, 192, 1, 1, weight_decay=weight_decay)

    x = merge([branch1x1, branch7x7, branch7x7dbl, branch_pool],
              mode='concat', concat_axis=channel_axis,
              name='mixed_7')

    if aux_include:
        aux_classifier = AveragePooling2D(
            (5, 5), strides=(3, 3), border_mode='valid')(x)
        aux_classifier = conv2d_bn(
            aux_classifier, 128, 1, 1, weight_decay=weight_decay)
        aux_classifier = conv2d_bn(aux_classifier, 768, 5, 5,
                                   border_mode='valid',
                                   weight_decay=weight_decay)

        aux_classifier = Flatten()(aux_classifier)

        if weight_decay and weight_decay > 0:
            aux_classifier = Dense(num_classes, activation='softmax',
                                   W_regularizer=l2(weight_decay),
                                   name='aux_classifier')(aux_classifier)
        else:
            aux_classifier = Dense(
                num_classes, activation='softmax',
                name='aux_classifier')(aux_classifier)

    # mixed 8: 8 x 8 x 1280.
    branch3x3 = conv2d_bn(x, 192, 1, 1, weight_decay=weight_decay)
    branch3x3 = conv2d_bn(branch3x3, 320, 3, 3,
                          subsample=(2, 2),
                          border_mode='valid',
                          weight_decay=weight_decay)

    branch7x7x3 = conv2d_bn(x, 192, 1, 1, weight_decay=weight_decay)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 1, 7, weight_decay=weight_decay)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 7, 1, weight_decay=weight_decay)
    branch7x7x3 = conv2d_bn(branch7x7x3, 192, 3, 3,
                            subsample=(2, 2),
                            border_mode='valid',
                            weight_decay=weight_decay)

    branch_pool = MaxPooling2D((3, 3), strides=(2, 2), border_mode='valid')(x)

    x = merge([branch3x3, branch7x7x3, branch_pool],
              mode='concat', concat_axis=channel_axis,
              name='mixed_8')

    # mixed 9 10: 8 x 8 x 2048
    for i in range(2):
        branch1x1 = conv2d_bn(x, 320, 1, 1, weight_decay=weight_decay)

        branch3x3 = conv2d_bn(x, 384, 1, 1, weight_decay=weight_decay)
        branch3x3 = merge([conv2d_bn(branch3x3, 384, 1, 3,
                                     weight_decay=weight_decay),
                           conv2d_bn(branch3x3, 384, 3, 1,
                                     weight_decay=weight_decay)],
                          mode='concat', concat_axis=channel_axis)

        branch3x3dbl = conv2d_bn(x, 448, 1, 1, weight_decay=weight_decay)
        branch3x3dbl = conv2d_bn(
            branch3x3dbl, 384, 3, 3, weight_decay=weight_decay)
        branch3x3dbl = merge([conv2d_bn(branch3x3dbl, 384, 1, 3,
                                        weight_decay=weight_decay),
                              conv2d_bn(branch3x3dbl, 384, 3, 1,
                                        weight_decay=weight_decay)],
                             mode='concat', concat_axis=channel_axis)

        branch_pool = AveragePooling2D(
            (3, 3), strides=(1, 1), border_mode='same')(x)
        branch_pool = conv2d_bn(
            branch_pool, 192, 1, 1, weight_decay=weight_decay)

        x = merge([branch1x1, branch3x3, branch3x3dbl, branch_pool],
                  mode='concat', concat_axis=channel_axis,
                  name='mixed_' + str(9 + i))

    # 2048 x 8 x 8
    x = conv2d_bn(x, 1024, 1, 1,
                  weight_decay=weight_decay)

    # 1024 x 8 x 8
    x = GlobalAveragePooling2D()(x)
    x = Dropout(dropout_prob)(x)

    # 1024
    if weight_decay and weight_decay > 0:
        predictions = Dense(num_classes,
                            activation='softmax',
                            W_regularizer=l2(weight_decay),
                            name='predictions')(x)
    else:
        predictions = Dense(num_classes,
                            activation='softmax',
                            name='predictions')(x)

    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input

    if aux_include:
        model = Model(
            inputs, [predictions, aux_classifier],
            name='inception_v3_with_aux')
    else:
        model = Model(inputs, predictions, name='inception_v3')

    return model

In [0]:
def load_data():
    (_, _), (x_train, y_train) = cifar10.load_data()
    x_train = x_train[:100]
    y_train = y_train[:100]
    print(x_train.shape)

    data_upscaled = np.zeros((100, 3, 299, 299))

    for i, img in enumerate(x_train):
        #im = img.transpose((1, 2, 0))
        large_img = cv2.resize(img, dsize=(299, 299), interpolation=cv2.INTER_CUBIC)
        #data_upscaled[i] = large_img.transpose((2, 0, 1))

    y_train = np_utils.to_categorical(y_train, 10)

    return data_upscaled, y_train

In [0]:
x_train, y_train = load_data()
model = Inception_V3(num_classes=10)
model.compile(optimizer='rmsprop',
                  loss={'predictions': 'categorical_crossentropy',
                        'aux_classifier': 'categorical_crossentropy'},
                  loss_weights={'predictions': 1., 'aux_classifier': 0.2})

(100, 32, 32, 3)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", name="conv_0", strides=(2, 2), padding="valid", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", name="conv_1", strides=(1, 1), padding="valid", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="conv_2", strides=(1, 1), padding="same", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(80, (1, 1), activation="relu", name="conv_3", strides=(1, 1), padding="same", kernel_regularizer=<keras.reg...)`
/usr/local/lib/pyt

TypeError: ignored

In [0]:
input_shape = x_train.shape[1:]
n_classes = 100

model = inception_v3(input_shape, n_classes)
sgd = optimizers.SGD(lr=0.045, decay=0.94, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
datagen = ImageDataGenerator(horizontal_flip=True, width_shift_range=0.1, height_shift_range=0.1)

datagen.fit(x_train)

model.fit_generator(datagen.flow(x_train, y_train, batch_size=64),
                    steps_per_epoch=x_train.shape[0] // 64,
                    epochs=60,
                    verbose=1,
                    validation_data=(x_test, y_test))

model.evaluate(x_test, y_test)